In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import glob

# We should consider filtering our data a bit because it's a bit messy currently

**I've written this scaffold to make things easier but code similar to what I've written below should be written as a function or two and run for every host and every virus probably saving as a new "tsv" file called "yadayada.clean.tsv"**

**This currently works on E. coli but it's possible that there will be some weird genomes that something in here screws up on so we'll need to think about double checking things to make sure that we never remove too much data. Which is to say that df.shape shouldn't change *that* dramatically from beginning to end and if it does it's likely that there is a bug in the code that needs to be investigated**

In [14]:
###Read in the starting TSV
df = pd.read_csv('../Data/562_rep_viruses/170.tsv', sep='\t', index_col=0)
###Reset the index so that it runs from zero to whatever
df = df.reset_index(drop=True)
print(df.shape)
df.head()
df["qualifiers"][1]

(290, 12)


'ID=357_2;partial=00;start_type=ATG;rbs_motif=AGGAG;rbs_spacer=5-10bp;gc_cont=0.353;conf=99.97;score=34.98;cscore=18.60;sscore=16.39;rscore=10.28;uscore=3.16;tscore=2.95;'

**First, make sure the table only uses each `locus_tag` once. These should be unique identifiers for the hosts and should therefore only appear once. If they appear more than once I have no idea what's happening. The virus dataframes won't be able to run this part of the code, however.**

In [19]:
df['locus_tag'] = df['qualifiers'].str.split('Parent=gene-', n=1, expand=True)[1]\
                            .str.split(';', n=1, expand=True)[0]

In [20]:
test_string = df.loc[0]["qualifiers"]

In [21]:
test_string

'ID=cds-WP_003262958.1;Parent=gene-RS_RS25220;Dbxref=Genbank:WP_003262958.1,GeneID:31479960;Name=WP_003262958.1;gbkey=CDS;gene=rpmH;inference=COORDINATES: similar to AA sequence:RefSeq:WP_003816025.1;locus_tag=RS_RS25220;product=50S ribosomal protein L34;protein_id=WP_003262958.1;transl_table=11'

In [22]:
df['locus_tag'].value_counts()[:10]

RS_RS07500    2
RS_RS07225    2
RS_RS25525    2
RS_RS07245    2
RS_RS11380    2
RS_RS04135    2
RS_RS26580    2
RS_RS25540    2
RS_RS09230    2
RS_RS26610    2
Name: locus_tag, dtype: int64

**Oooof, let's get rid of those**

In [23]:
df = df.drop_duplicates(subset=['locus_tag'], keep=False)
print(df.shape)

(3432, 13)


In [24]:
df['locus_tag'].value_counts()[:10]

RS_RS11035    1
RS_RS08945    1
RS_RS16780    1
RS_RS00940    1
RS_RS14325    1
RS_RS10380    1
RS_RS06340    1
RS_RS25705    1
RS_RS00770    1
RS_RS16440    1
Name: locus_tag, dtype: int64

**Voila!**

**How about any weird length genes? This should be checked and removed for all genomes (host and virus)**

In [25]:
((df['stop']-df['start']) %3).value_counts()

2    3385
1      24
0      23
dtype: int64

**Clean them up too**

In [26]:
df = df[(df['stop']-df['start']) %3 == 2]
print(df.shape)

(3385, 13)


**Remove phage/prophage genes from descriptors. This again should only be run for the host genomes so you can see maybe we'll want separate functions like "def clean_host(...)" and "def clean_virus(...)"**

In [30]:
filter_word = 'phage'
# filter_word = 'phage'
virus_genes = df[(df['qualifiers'].str.contains(filter_word)==True)]
print(virus_genes.shape)

(31, 13)


In [34]:
df = df[(df['qualifiers'].str.contains(filter_word)==False)]
df.shape

(3354, 13)

0.9676860934795153

In [33]:
virus_genes.loc[93]["qualifiers"]

'ID=cds-WP_011000059.1;Parent=gene-RS_RS00465;Dbxref=Genbank:WP_011000059.1,GeneID:1218895;Name=WP_011000059.1;gbkey=CDS;inference=COORDINATES: similar to AA sequence:RefSeq:WP_003261510.1;locus_tag=RS_RS00465;product=phage holin family protein;protein_id=WP_011000059.1;transl_table=11'

# Add in knowledge of codon usage bias

** This is an important control and you'll have to install this "iCUB" software to make this run. I worked on this project with a former graduate student in my PhD lab and the code is here: https://github.com/amarallab/iCUB **

**The major bug that I can anticipate cropping up is if the organism uses a "non-standard" genetic code but we'll cross that bridge when we get there. This will also probably cause an error if any "non-standard" nucleotides appear in a given genome which will almost certainly occur and need to be fixed**

In [6]:
import sys
sys.path.append('/Users/ChaseWeaver/Desktop/Projects/iCUB-master/')
import iCUB

In [84]:
for index in df.index:
    nt_seq = df.at[index, "coding_sequence"]
    if len(nt_seq)%3 == 0:
        df.at[index, "iCUB"] = iCUB.iCUB_Calculator(nt_seq).get_iCUB()

KeyboardInterrupt: 

**If our genes are clean and good then we shouldn't have any null values in either of these categories and we can then write this new dataframe**

In [85]:
print(df[df['iCUB'].isnull()==True].shape)
print(df[df['energy_binding'].isnull()==True].shape)

(3820, 14)
(0, 14)


# Final notes:

**As I mentioned, we aren't going to want to step through this code for every genome so your task is to write this up into some functions that you can then put into for loops and run for everyone. I'm imagining something like:**

<code>
for host_tsv_file in glob.glob('blahblah/\*.tsv): 
    clean_host_tsv(host_tsv_file)
</code>
    
**where `clean_host_tsv()` will read the tsv, clean it up, check that no problems occurred, and write the new tsv. Something along the lines of:**

<code>
def clean_host_tsv(tsv_location):

    df = pd.read_tsv(tsv_location...)
    starting_shape = df.shape
    #Code that does the work goes here
    ending_shape = df.shape
    #Code that compares ending and starting shapes to make sure things look reasonable goes here
    df.to_csv(tsv_location.replace('.tsv', '.clean.tsv'))
</code>


**And then again we'll have something like this for the viruses:**

<code>
for virus_folder in glob.glob('blahblah/\*\_rep\_viruses/'):
    for virus_tsv_file in glob.glob(virus_folder + '\*.tsv'):
        clean_virus_tsv(virus_tsv_file)
</code>


In [ ]:
df = pd.read_csv('../Data/host_genomes/562.tsv', sep = "\t", index_col = 0)
print(df.shape)

In [4]:
def clean_host_tsv(df):
    """
    
    """
    
    initial_shape = df.shape
    df = df.reset_index(drop = True)
    
    #Ensure each locus tag is only used once
    df['locus_tag'] = df['qualifiers'].str.split('locus_tag=', n=1, expand=True)[1]\
                            .str.split(';', n=1, expand=True)[0]
    df = df.drop_duplicates(subset = ["locus_tag"], keep = False)
    
    #ensure that only genes with a total length divisible by 3 are used
    df = df[(df['stop']-df['start']) %3 == 2]
    
    #filter out prophages from dataframe
    filter_word = 'phage'
    virus_genes = df[(df['qualifiers'].str.contains(filter_word)==True)]
    df = df[(df['qualifiers'].str.contains(filter_word)==False)]
    
    #adjust for codon bias
    for index in df.index:
        nt_seq = df.at[index, "coding_sequence"]
        if len(nt_seq)%3 == 0:
            df.at[index, "iCUB"] = iCUB.iCUB_Calculator(nt_seq).get_iCUB()
    
    
    
    
    
    return df

In [3]:
for tsv_location in glob.glob("../Data/host_genomes/*.tsv"):
    print(tsv_location)

../Data/host_genomes/28450.tsv
../Data/host_genomes/357276.clean.tsv
../Data/host_genomes/1590.tsv
../Data/host_genomes/562.tsv
../Data/host_genomes/357276.tsv
../Data/host_genomes/657318.tsv
../Data/host_genomes/573.tsv
../Data/host_genomes/1280.clean.tsv
../Data/host_genomes/28450.clean.tsv
../Data/host_genomes/562.clean.tsv
../Data/host_genomes/1280.tsv
../Data/host_genomes/435591.clean.tsv
../Data/host_genomes/305.tsv
../Data/host_genomes/28450.1.tsv
../Data/host_genomes/28450.2.tsv
../Data/host_genomes/717959.clean.tsv
../Data/host_genomes/435591.tsv
../Data/host_genomes/470.tsv
../Data/host_genomes/1590.clean.tsv
../Data/host_genomes/287.tsv
../Data/host_genomes/1314.tsv
../Data/host_genomes/36809.tsv
../Data/host_genomes/717959.tsv
../Data/host_genomes/470.clean.tsv
../Data/host_genomes/1314.clean.tsv
../Data/host_genomes/90371.tsv
../Data/host_genomes/1639.clean.tsv
../Data/host_genomes/1639.tsv
../Data/host_genomes/90371.clean.tsv
../Data/host_genomes/305.clean.tsv
../Data/hos

In [7]:
for tsv_location in glob.glob("../Data/host_genomes/*.tsv"):
    if "clean" in tsv_location:
        continue
    df = pd.read_csv(tsv_location, sep = "\t", index_col = 0)
    initial_shape = df.shape
    df = clean_host_tsv(df)
    final_shape = df.shape
    gene_ratio = final_shape[0]/initial_shape[0]
    if gene_ratio <= .8 or gene_ratio == 1:
        print(tsv_location)
        break
    
    clean_tsv_loc = tsv_location.replace(".tsv",".clean.tsv")
    df.to_csv(clean_tsv_loc, sep = "\t")
    


In [28]:
df['qualifiers'].str.split('ID=')

0       [, cds-NP_414542.1;Parent=gene-b0001;Dbxref=Un...
1       [, cds-NP_414543.1;Parent=gene-b0002;Dbxref=Un...
2       [, cds-NP_414544.1;Parent=gene-b0003;Dbxref=Un...
3       [, cds-NP_414545.1;Parent=gene-b0004;Dbxref=Un...
4       [, cds-NP_414546.1;Parent=gene-b0005;Dbxref=Un...
5       [, cds-NP_414547.1;Parent=gene-b0006;Dbxref=Un...
6       [, cds-NP_414548.1;Parent=gene-b0007;Dbxref=Un...
7       [, cds-NP_414549.1;Parent=gene-b0008;Dbxref=Un...
8       [, cds-NP_414550.1;Parent=gene-b0009;Dbxref=Un...
9       [, cds-NP_414551.1;Parent=gene-b0010;Dbxref=Un...
10      [, cds-NP_414552.1;Parent=gene-b0011;Dbxref=Un...
11      [, cds-YP_009518733.1;Parent=gene-b0012;Dbxref...
12      [, cds-NP_414554.1;Parent=gene-b0013;Dbxref=Un...
13      [, cds-NP_414555.1;Parent=gene-b0014;Dbxref=Un...
14      [, cds-NP_414556.1;Parent=gene-b0015;Dbxref=Un...
15      [, cds-NP_414557.1;Parent=gene-b0016;Dbxref=Un...
16      [, cds-NP_414559.1;Parent=gene-b0018;Dbxref=Un...
17      [, cds

In [41]:
tsv_location ="../Data/host_genomes/717959.tsv"
     
df = pd.read_csv(tsv_location, sep = "\t", index_col = 0)
initial_shape = df.shape
df = clean_host_tsv(df)
final_shape = df.shape
gene_ratio = final_shape[0]/initial_shape[0]
if gene_ratio <= .8 or gene_ratio == 1:
    print(tsv_location)
    #break

clean_tsv_loc = tsv_location.replace(".tsv",".clean.tsv")
df.to_csv(clean_tsv_loc, sep = "\t")

In [31]:
print(gene_ratio)

0.9971290148932352


In [22]:
df['viral_id'] = df['qualifiers'].str.split('ID=', n=1, expand=True)[1]\
                            .str.split(';', n=1, expand=True)[0]

In [23]:
df.head()

,genome_id,source,type,start,stop,idk,strand,trash,qualifiers,coding_sequence,upstream_sequence,energy_binding,id,viral_id
0,NC_028957|1567029,Prodigal_v2.6.3,CDS,1,2235,272.7,-,0,ID=357_1;partial=00;start_type=ATG;rbs_motif=G...,ATGATTTTAGAAACTGAAAAAGAAACAATCCTTGGTAATGGCGCAA...,TAATATAAAGTGAGAAAAAC,-3.10,357_1,357_1
1,NC_028957|1567029,Prodigal_v2.6.3,CDS,2243,2446,35.0,-,0,ID=357_2;partial=00;start_type=ATG;rbs_motif=A...,ATGACAAACTTCAAAGTGAATCTGGAGCTGTTTGACAAGGTCGTTC...,AATATTGCAGGAGATTGGAA,-7.00,357_2,357_2
2,NC_028957|1567029,Prodigal_v2.6.3,CDS,2412,2711,24.4,-,0,ID=357_3;partial=00;start_type=ATG;rbs_motif=G...,ATGATTATGTCTGACGGAAGTTTAGTTAATCTGCAGAATGTCGCGC...,GATGTCCTTGAGGTGAAAAT,-5.13,357_3,357_3
3,NC_028957|1567029,Prodigal_v2.6.3,CDS,2787,2939,17.3,-,0,ID=357_4;partial=00;start_type=ATG;rbs_motif=A...,ATGAAAAAGTGGTGGGTTACGTTGTCAAATGGCGAATACGGTTATC...,CCGTCTTATAGGAGAAGTCC,-7.00,357_4,357_4
4,NC_028957|1567029,Prodigal_v2.6.3,CDS,2982,4814,225.2,-,0,ID=357_5;partial=00;start_type=ATG;rbs_motif=A...,ATGATTGAAAATAAAATTGAACTGCTTTCAGATAAAGACCATTGTT...,AAAGTAAGGAATAATTGATT,-4.24,357_5,357_5


In [25]:
def clean_virus_tsv(df):
    """
    
    """
    
    initial_shape = df.shape
    df = df.reset_index(drop = True)
    
    #Ensure each locus tag is only used once
    df['viral_id'] = df['qualifiers'].str.split('ID=', n=1, expand=True)[1]\
                            .str.split(';', n=1, expand=True)[0]
    df = df.drop_duplicates(subset = ["viral_id"], keep = False)
    
    #ensure that only genes with a total length divisible by 3 are used
    df = df[(df['stop']-df['start']) %3 == 2]
    
    #filter out prophages from dataframe
#     filter_word = 'phage'
#     virus_genes = df[(df['qualifiers'].str.contains(filter_word)==True)]
#     df = df[(df['qualifiers'].str.contains(filter_word)==False)]
    
    #adjust for codon bias
    for index in df.index:
        nt_seq = df.at[index, "coding_sequence"]
        if len(nt_seq)%3 == 0:
            df.at[index, "iCUB"] = iCUB.iCUB_Calculator(nt_seq).get_iCUB()
    
    
    
    
    
    return df